# aT 농산품 예측 base line

&nbsp;

## 모듈 가져오기

In [194]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit,njit

test_num = 7

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

## 입력 shape 및 형태 정의 함수

In [195]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [196]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [197]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [198]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=50)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [199]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.002, verbose = 1):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [200]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [201]:
data_list = glob('./aT_data/data/train/*.csv')
epoch = 1000
batch = 64
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

&nbsp;

## Train 과정

In [202]:
for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch)
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

100%|██████████| 1377/1377 [00:00<00:00, 80437.27it/s]


Epoch 1/1000


2022-09-01 21:26:54.049845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/17 [===============================] - ETA: 0s - loss: 19073826816.0000

2022-09-01 21:26:57.720903: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1072140864.00000, saving model to ./check/transformer-4-1000-64.h5
17/17 [==============================] - 8s 128ms/step - loss: 19073826816.0000 - val_loss: 1072140864.0000
Epoch 2/1000
18/17 [===============================] - ETA: 0s - loss: 1244097664.0000
Epoch 2: val_loss improved from 1072140864.00000 to 250648288.00000, saving model to ./check/transformer-4-1000-64.h5
17/17 [==============================] - 0s 10ms/step - loss: 1244097664.0000 - val_loss: 250648288.0000
Epoch 3/1000
14/17 [=======================>......] - ETA: 0s - loss: 334866080.0000
Epoch 3: val_loss improved from 250648288.00000 to 140153056.00000, saving model to ./check/transformer-4-1000-64.h5
17/17 [==============================] - 0s 7ms/step - loss: 411579136.0000 - val_loss: 140153056.0000
Epoch 4/1000
13/17 [=====================>........] - ETA: 0s - loss: 183691248.0000
Epoch 4: val_loss improved from 140153056.00000 to 102242568.00000, saving model to .

  0%|          | 0/37 [00:19<?, ?it/s]


KeyboardInterrupt: 

&nbsp;

## Test 과정

In [ ]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./aT_data/data/test/set_{i}/*.csv')
    
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        print(df.shape)
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]
    
        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists(f'./model_output/{test_num}') == False:
            os.mkdir(f'./model_output/{test_num}')

        if os.path.exists(f'./model_output/{test_num}/set_{i}') == False:
            os.mkdir(f'./model_output/{test_num}/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/{test_num}/set_{i}/predict_{file_number}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: in user code:

    File "/Users/chai/miniforge3/envs/nongnet/lib/python3.8/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/chai/miniforge3/envs/nongnet/lib/python3.8/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/chai/miniforge3/envs/nongnet/lib/python3.8/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/chai/miniforge3/envs/nongnet/lib/python3.8/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/chai/miniforge3/envs/nongnet/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/chai/miniforge3/envs/nongnet/lib/python3.8/sit

&nbsp;

## 정답 제출 파일생성

In [ ]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/{test_num}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:01<00:00,  7.57it/s]


- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.1250,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.2800,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,4368.9185,1136.835200,4341.878400,1304.011800,358.825800,2668.822300,1291.3479,3296.437300,3038.088600,580.204650,...,7152.747000,742.189100,2.857842,60.241528,3475.514600,2496.694000,2835.005100,1771.467900,1454.097900,1337.244500
2,3842.6377,540.509160,3001.460000,1038.376800,375.114800,2660.001700,1297.8126,2354.604700,2898.331800,418.600500,...,4749.454600,598.127000,3.852921,62.373806,3295.248800,2360.204800,2455.846200,1768.950100,1387.853000,1345.857800
3,1384.4365,453.866580,1992.754600,239.199280,464.757080,2695.452100,1296.0669,1406.715100,-782.994260,-35.508460,...,2141.611300,719.498500,3.023891,62.653458,221.171880,2472.923600,-351.211200,1714.565100,363.272250,1338.831800
4,4350.0283,1082.508000,4689.017000,1084.178000,516.913100,2667.023200,1260.0607,3681.917700,2595.567000,194.602160,...,7437.019500,687.689150,5.440804,61.029724,3806.945300,2611.558300,3065.139000,1785.715500,2061.381300,1386.378900
5,4315.9927,1086.238900,4659.872000,985.549400,464.616360,2665.436500,1282.8461,3874.471400,2676.024200,246.440380,...,7231.182600,706.584500,2.519975,60.551113,3494.673300,2302.620000,2848.984100,1810.414400,2000.205300,1377.335300
6,4286.0654,1122.811200,4779.183000,1290.911000,390.804050,2646.664300,1284.7887,3568.861800,3088.690200,377.151520,...,7573.511000,757.412800,4.436273,59.689780,3581.851000,2389.863300,2972.006800,1816.999300,2002.211900,1361.995500
7,4074.2370,1156.785500,4570.998000,1169.737900,453.829100,2670.284200,1281.4319,3431.533700,2186.527000,-124.446160,...,7091.532700,710.473270,4.232679,60.470673,3621.887500,2397.004000,2894.915000,1758.072500,2003.286500,1342.896500
8,4184.6090,1122.200700,4371.313500,1217.054700,511.346700,2679.619100,1245.5442,3029.554200,2655.349900,820.545100,...,6458.230500,780.316900,6.201695,62.063988,3649.317600,2466.483600,2770.797400,1736.283100,1673.523700,1327.816700
9,3908.3127,539.375200,2955.974900,945.003800,486.983060,2660.305000,1280.1942,2300.175300,3033.382800,556.706050,...,3630.171000,518.340500,6.503197,57.144833,3560.594700,2380.364700,2453.345000,1797.134300,1622.239300,1347.760500


- 변동률 계산 

In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [ ]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,0.128591,-0.165391,0.492165,-0.616476,-0.909108,-0.711602,-0.524765,-0.019218,-0.381484,-0.505375,...,-0.997415,-0.961538,-0.043471,-0.542287,-0.495478,-0.661327,-0.499467,-0.359458,0,d+1
1,-0.007359,-0.603185,0.031506,-0.694602,-0.904982,-0.712555,-0.522385,-0.299440,-0.409937,-0.643142,...,-0.996514,-0.960177,-0.093084,-0.567309,-0.562954,-0.661809,-0.522270,-0.355332,0,d+2
2,-0.642368,-0.666793,-0.315154,-0.929649,-0.882275,-0.708724,-0.523028,-0.581463,-1.159408,-1.030271,...,-0.997264,-0.959998,-0.939129,-0.546644,-1.062502,-0.672206,-0.874954,-0.358697,0,d+3
3,0.123712,-0.205276,0.611466,-0.681131,-0.869063,-0.711796,-0.536279,0.095473,-0.471576,-0.834101,...,-0.995078,-0.961035,0.047745,-0.521229,-0.454523,-0.658603,-0.290427,-0.335922,0,d+4
4,0.114919,-0.202537,0.601450,-0.710139,-0.882310,-0.711968,-0.527893,0.152763,-0.455196,-0.789909,...,-0.997720,-0.961341,-0.038198,-0.577866,-0.492990,-0.653881,-0.311485,-0.340254,0,d+5
5,0.107189,-0.175687,0.642453,-0.620329,-0.901007,-0.713996,-0.527178,0.061835,-0.371182,-0.678478,...,-0.995987,-0.961891,-0.014205,-0.561872,-0.471097,-0.652623,-0.310794,-0.347602,0,d+6
6,0.052468,-0.150745,0.570906,-0.655967,-0.885043,-0.711444,-0.528414,0.020977,-0.554851,-1.106091,...,-0.996171,-0.961392,-0.003186,-0.560563,-0.484816,-0.663888,-0.310424,-0.356750,0,d+7
7,0.080980,-0.176135,0.502281,-0.642051,-0.870473,-0.710435,-0.541621,-0.098624,-0.459405,-0.300484,...,-0.994390,-0.960375,0.004363,-0.547825,-0.506904,-0.668054,-0.423936,-0.363973,0,d+8
8,0.009606,-0.604017,0.015874,-0.722064,-0.876645,-0.712522,-0.528869,-0.315634,-0.382442,-0.525407,...,-0.994117,-0.963515,-0.020055,-0.563613,-0.563399,-0.656420,-0.441589,-0.354420,0,d+9
9,-0.641542,-0.686545,-0.388035,-0.900212,-0.892334,-0.710016,-0.522573,-0.560002,-1.100108,-0.148812,...,-0.994118,-0.961480,-0.864663,-0.558366,-1.058027,-0.660981,-0.890847,-0.351731,0,d+10


- 정답 양식으로 변경

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit/submit_{test_num}.csv',index=False)

- 계산된 변동률 결과물

In [ ]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.128591,-0.165391,0.492165,-0.616476,-0.909108,-0.711602,-0.524765,-0.019218,...,-0.172213,0.232861,-0.997415,-0.961538,-0.043471,-0.542287,-0.495478,-0.661327,-0.499467,-0.359458
1,0,d+2,-0.007359,-0.603185,0.031506,-0.694602,-0.904982,-0.712555,-0.522385,-0.299440,...,-0.450346,-0.006443,-0.996514,-0.960177,-0.093084,-0.567309,-0.562954,-0.661809,-0.522270,-0.355332
2,0,d+3,-0.642368,-0.666793,-0.315154,-0.929649,-0.882275,-0.708724,-0.523028,-0.581463,...,-0.752152,0.195169,-0.997264,-0.959998,-0.939129,-0.546644,-1.062502,-0.672206,-0.874954,-0.358697
3,0,d+4,0.123712,-0.205276,0.611466,-0.681131,-0.869063,-0.711796,-0.536279,0.095473,...,-0.139315,0.142330,-0.995078,-0.961035,0.047745,-0.521229,-0.454523,-0.658603,-0.290427,-0.335922
4,0,d+5,0.114919,-0.202537,0.601450,-0.710139,-0.882310,-0.711968,-0.527893,0.152763,...,-0.163136,0.173717,-0.997720,-0.961341,-0.038198,-0.577866,-0.492990,-0.653881,-0.311485,-0.340254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.297328,-0.511580,0.245290,-0.195912,-0.979511,-0.787249,-0.838971,NaN,...,0.044825,-0.573435,-0.995014,-1.667253,0.064671,-0.504821,-0.599455,0.146352,-0.069956,-0.483879
146,9,d+12,-0.283990,-0.523207,0.120178,-0.230529,-0.983976,-0.788503,-0.836559,NaN,...,0.042968,-0.579935,-0.995308,0.119258,0.038526,-0.484822,-0.589613,0.120131,-0.073522,-0.486060
147,9,d+13,-0.270182,-0.473874,0.184459,-0.221874,-0.983642,-0.787669,-0.847669,NaN,...,0.007669,-0.545746,-0.997112,-0.269711,-0.063759,-0.548052,-0.584868,0.126781,-0.120936,-0.471455
148,9,d+14,-0.241916,-0.526987,0.190409,-0.199000,-0.981041,-0.786176,-0.811106,NaN,...,0.031082,-0.546494,-0.996512,-0.572904,0.024094,-0.544563,-0.653693,0.144321,-0.126116,-0.474149


- 제출 양식

In [ ]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [ ]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.128591,-0.165391,0.492165,-0.616476,-0.909108,-0.711602,-0.524765,-0.019218,...,-0.172213,0.232861,-0.997415,-0.961538,-0.043471,-0.542287,-0.495478,-0.661327,-0.499467,-0.359458
1,0,d+2,-0.007359,-0.603185,0.031506,-0.694602,-0.904982,-0.712555,NaN,-0.299440,...,-0.450346,-0.006443,-0.996514,-0.960177,-0.093084,-0.567309,-0.562954,-0.661809,-0.522270,-0.355332
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.523028,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.123712,-0.205276,0.611466,-0.681131,-0.869063,-0.711796,NaN,0.095473,...,-0.139315,0.142330,-0.995078,-0.961035,0.047745,-0.521229,-0.454523,-0.658603,-0.290427,-0.335922
4,0,d+5,0.114919,-0.202537,0.601450,-0.710139,-0.882310,-0.711968,NaN,0.152763,...,-0.163136,0.173717,-0.997720,-0.961341,-0.038198,-0.577866,-0.492990,-0.653881,-0.311485,-0.340254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,-0.297328,-0.511580,0.245290,-0.195912,-0.979511,-0.787249,-0.838971,NaN,...,0.044825,-0.573435,-0.995014,-1.667253,0.064671,-0.504821,-0.599455,0.146352,-0.069956,-0.483879
146,9,d+12,-0.283990,-0.523207,0.120178,-0.230529,-0.983976,-0.788503,-0.836559,NaN,...,0.042968,-0.579935,-0.995308,0.119258,0.038526,-0.484822,-0.589613,0.120131,-0.073522,-0.486060
147,9,d+13,-0.270182,-0.473874,0.184459,-0.221874,-0.983642,-0.787669,NaN,NaN,...,0.007669,-0.545746,-0.997112,-0.269711,-0.063759,-0.548052,-0.584868,0.126781,-0.120936,-0.471455
148,9,d+14,-0.241916,-0.526987,0.190409,-0.199000,-0.981041,-0.786176,-0.811106,NaN,...,0.031082,-0.546494,-0.996512,-0.572904,0.024094,-0.544563,-0.653693,0.144321,-0.126116,-0.474149
